# Imports

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import pinecone
import openai
from langchain.document_loaders import DirectoryLoader, Docx2txtLoader, TextLoader
from langchain.vectorstores import Pinecone


/home/ubuntu/.pyenv/versions/langchain-3.10.2/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
from langchain.document_loaders import Docx2txtLoader

# PAths and creds

In [5]:
import sys
sys.path.append('/home/ubuntu/workspace/Creds')
from openai_config import OPENAI_API_KEY, MW_ENVIRONMENT, MW_PINECONE_API_KEY
openai.api_key = OPENAI_API_KEY

In [6]:
embeddings = OpenAIEmbeddings(model = 'text-embedding-ada-002',
                              openai_api_key= OPENAI_API_KEY)

In [7]:
pinecone_mw_index_name = 'minimal-index-dynamic-chatbot'

In [8]:
pinecone.init(
    api_key= MW_PINECONE_API_KEY,  # find at app.pinecone.io
    environment= MW_ENVIRONMENT,  # next to api key in console
)

# Fns, and variables

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["#","##", "###", "\\n\\n","\\n",".", '\n'],
    chunk_size=1500,
    chunk_overlap=100)

In [ ]:
def return_docs_from_pinecone_raw(pinecone_index, query, embeddings =embeddings):


    '''
    will return top 3 relevant chunks, and also top one, for decision making
    '''
    query_vector = embeddings.embed_query(query)
    docs = pinecone_index.query(query_vector,top_k=3,include_metadata=True)
    contexts = [
        x['metadata']['text'] for x in docs['matches']
    ]
    final_str = ''
    for one in contexts:
        final_str += one
        final_str +='\n\n\n\n'

    return final_str, contexts[0]
    

In [60]:
def return_docs_from_pinecone_lang(pinecone_index, query, embeddings =embeddings):

    '''
    will return top 3 relevant chunks, and also top one, for decision making
    '''
    
    docs = pinecone_index.similarity_search(query, k =3)

    final_str = ''
    for one in docs:
        final_str += one.page_content
        final_str +='\n\n\n\n'

    return final_str, docs[0].page_content
    

In [52]:
def update_pinecone_texts(pinecone_index, texts_to_add, index_name):
    
    #index_name = pinecone_index.configuration.server_variables['index_name']
    updated_index = Pinecone.from_texts(texts_to_add,embeddings, index_name = index_name)
    return updated_index
def update_pinecone_docs(pinecone_index, docs_to_add, index_name):

    #index_name = pinecone_index.configuration.server_variables['index_name']
    updated_index = Pinecone.from_documents(docs_to_add,embeddings, index_name = index_name)
    return updated_index

# Work

## upserting the index

In [83]:
docx_doc = Docx2txtLoader('/home/ubuntu/workspace/mrwhisper-codespace/ChatBot-Work/web_app/knowledgebase/resolve-readme-summary.docx')


In [84]:
documents = docx_doc.load()

In [85]:
docs = text_splitter.split_documents(documents)

In [86]:
pinecone.list_indexes()

['minimal-index-dynamic-chatbot', 'mw-index-1']

In [87]:
len(docs)

74

In [88]:
docsearch = Pinecone.from_documents(docs, embeddings, index_name=pinecone_mw_index_name)


### Full workflow with pinecone, depiction

In [11]:
sys.path.append('/home/ubuntu/workspace/mrwhisper-codespace/ChatBot-Work/trial_work')

In [14]:
from plain_help import *

In [43]:
docsearch = Pinecone.get_pinecone_index(pinecone_mw_index_name)

In [16]:
update_pinecone_docs, update_pinecone_texts

(<function plain_help.update_pinecone_docs(pinecone_index, docs_to_add, index_name)>,
 <function plain_help.update_pinecone_texts(pinecone_index, texts_to_add, index_name)>)

# RoughWork

In [28]:
docsearch.configuration.server_variables['index_name']

'minimal-index-dynamic-chatbot'

In [40]:
docsearch.configuration.__dict__

AttributeError: 'Pinecone' object has no attribute 'configuration'

In [33]:
docsearch.delete(delete_all=True)

{}

In [53]:
updated_index = update_pinecone_docs(docsearch,docs,pinecone_mw_index_name)

In [62]:
docs, top_doc = return_docs_from_pinecone_lang(updated_index,'resolve')
len(docs)

5947

In [45]:
updated_index

In [51]:
updated_index.get_pinecone_index()

TypeError: Pinecone.get_pinecone_index() missing 1 required positional argument: 'index_name'

In [63]:
a = updated_index.similarity_search('Resolve',)
len(a)

4

In [82]:
updated_index.delete(delete_all=True)

In [72]:
pinecone_index = Pinecone.get_pinecone_index(pinecone_mw_index_name)


In [74]:
aa = Pinecone.from_existing_index(pinecone_mw_index_name,embeddings)

In [75]:
aa.similarity_search('What is Fiftyone')

[Document(page_content='# What is the latest version of FiftyOne by Voxel51?\n\n## ⚡ Quick Search\n1. [docs.voxel51.com - FiftyOne Release Notes — FiftyOne 0.21.6 documentation](https://docs.voxel51.com/release-notes.html)\n2. [voxel51.com - FiftyOne Open Source - Voxel51](https://voxel51.com/fiftyone/)\n3. [docs.voxel51.com - FiftyOne — FiftyOne 0.21.6 documentation](https://docs.voxel51.com/)\n4. [docs.voxel51.com - FiftyOne Installation — FiftyOne 0.21.6 documentation - Voxel51](https://docs.voxel51.com/getting_started/install.html)\n5. [voxel51.com - Announcing FiftyOne 0.21 with Operators, Dynamic Groups ...](https://voxel51.com/blog/announcing-fiftyone-0-21/)\n6. [voxel51.com - Announcing FiftyOne 0.20 with Natural Language Search, ...](https://voxel51.com/blog/announcing-fiftyone-0-20/)\n\n#', metadata={'source': '/home/ubuntu/workspace/Temp/UPLOADED-DIR/JSON-Documents-20231007-191713/conversations.json'}),
 Document(page_content=".3.5. Released August 7, 2023. Includes all upda